In [3]:
import numpy
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import date, datetime
from multiprocessing.spawn import import_main_path
from keras.preprocessing.sequence import TimeseriesGenerator
from pandas_datareader import data
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np



In [4]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset



In [5]:
tickers = ["ANTM.JK","ASII.JK"]

today = date.today()
# print(today)
start_date = '2010-01-01'
end_date = today

panel_data = data.DataReader(tickers, 'yahoo',start_date, end_date)


In [6]:

close = panel_data["Close"]

all_weekdays = pd.date_range(start = start_date,    end= end_date, freq='B')

close = close.reindex(all_weekdays)

close = close.dropna()

#print(all_weekdays)

In [7]:
close

Symbols,ANTM.JK,ASII.JK
2010-01-04,1931.946777,3530.0
2010-01-05,1931.946777,3550.0
2010-01-06,1931.946777,3530.0
2010-01-07,1973.945557,3420.0
2010-01-08,1973.945557,3440.0
...,...,...
2022-02-11,1845.000000,5450.0
2022-02-14,1890.000000,5300.0
2022-02-15,1880.000000,5500.0
2022-02-16,1880.000000,5475.0


In [8]:
dataframe = close
dataset = dataframe.values
dataset = dataset.astype('float32')

In [36]:
# normalize the dataset
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


In [29]:
# reshape input to be [samples, time steps, features]
trainX_rs = trainX.reshape(trainX.shape[0], trainX.shape[1],1)
testX_rs = testX.reshape(testX.shape[0], trainX.shape[1],1)

In [33]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX_rs, trainY, epochs=5, batch_size=1, verbose=2)


Epoch 1/5
2013/2013 - 5s - loss: 0.0119 - 5s/epoch - 3ms/step
Epoch 2/5
2013/2013 - 4s - loss: 1.9187e-04 - 4s/epoch - 2ms/step
Epoch 3/5
2013/2013 - 4s - loss: 1.8454e-04 - 4s/epoch - 2ms/step
Epoch 4/5
2013/2013 - 4s - loss: 1.7668e-04 - 4s/epoch - 2ms/step
Epoch 5/5
2013/2013 - 4s - loss: 1.7151e-04 - 4s/epoch - 2ms/step


In [37]:
# make predictions
trainPredict = model.predict(trainX_rs)
testPredict = model.predict(testX_rs)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

ValueError: non-broadcastable output operand with shape (2013,1) doesn't match the broadcast shape (2013,2)

In [12]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

TypeError: Singleton array 0.5811042 cannot be considered a valid collection.